### https://radimrehurek.com/gensim/models/doc2vec.html


In [1]:
import gensim
from gensim import utils

import random
import os
from collections import Counter
from pprint import pprint
import numpy as np
import pandas as pd
from tqdm import trange

from imblearn.combine import SMOTEENN 
from imblearn.over_sampling import RandomOverSampler,SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import make_pipeline as make_pipeline_imb
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import ADASYN 
from imblearn.ensemble import BalanceCascade
import imblearn

import nltk.stem as stem
from nltk.tokenize import word_tokenize
from nltk.tokenize import wordpunct_tokenize

from sklearn.ensemble import RandomForestClassifier
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.cross_validation import cross_val_score

/home/enxie/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
df_finn = pd.read_csv('./data/ebay2gg_table')
#df_finn

In [3]:
df_gg2id = pd.read_csv('./data/gpc_id2name.tsv',sep="\t")
#df_gg2id

In [4]:
gg_cate = list(df_gg2id['GPC_NAME'])

sql = 'GPC_NAME=="need_replace"'
'''
X是ebay cate，如 Jewelry & Watches:Vintage & Antique Jewelry:Fine:Designer, Signed:Rings
y是对应google cate的label(类别),在0-1997之间的数
d是一个dict，key为label值,value为对应的google cate的名字 如  {0: 'Animals & Pet Supplies'}
'''
X=[]
y=[]
d=[]

label=0
for i in trange(len(gg_cate)):
    new_sql = sql.replace("need_replace",gg_cate[i])
    res_list = list(df_finn.query(new_sql)['leaf_categ_name'])
    #如果样本数量大于k
    if len(res_list)>0:
        d.append([label,gg_cate[i]])
        for j in res_list:
            X.append(j)
            y.append(label)
        label+=1
d=dict(d)
# d

100%|██████████| 5427/5427 [00:17<00:00, 317.76it/s]


In [5]:
'''打乱数据，分割训练，验证集，可以设置按每一类比例随机抽（stratify =y）'''
X,y=shuffle(X,y)
#X_train , X_test , y_train  , y_test = train_test_split(X,y,test_size=0.2,stratify =y) 
X_train , X_test , y_train  , y_test = train_test_split(X,y,test_size=0.3)

In [ ]:
print(len(X_train),len(X_test))


26366 11300


In [ ]:
'''
构建pipeline，包括了tfidf抽取特征和 rf分类
'''
pipeline = make_pipeline(TfidfVectorizer(),RandomForestClassifier(n_estimators=20,n_jobs=8))

# pipeline = make_pipeline_imb(TfidfVectorizer(),
#                          #ADASYN(n_neighbors=2,n_jobs=4),
#                          #RandomOverSampler(),
#                          SMOTE(random_state=0,n_jobs=4,k_neighbors=2),
#                              #SMOTEENN(random_state=42),
#                             # BalanceCascade(random_state=42)
#                          RandomForestClassifier(n_estimators=20,n_jobs=4))

'''训练模型'''
pipeline.fit(X_train, y_train)
'''交叉验证'''
scores = cross_val_score(pipeline, X_train, y_train,n_jobs=2,cv=5,verbose=1)
print(scores.mean())

/home/enxie/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:553: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=5.
  % (min_labels, self.n_folds)), Warning)


In [ ]:
'''模型的准确率'''
pipeline.score(X_test,y_test)

In [30]:
'''
模型在验证集上的性能评估(准确率，召回率，f1指标和样本数)
'''
from sklearn.metrics import classification_report
y_pred = pipeline.predict(X_test)
# print(classification_report_imbalanced(y_test, y_pred))
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.67      0.18      0.29        11
          1       0.00      0.00      0.00         2
          2       0.00      0.00      0.00         1
          3       0.00      0.00      0.00         1
          4       0.50      0.50      0.50         2
          5       0.60      0.60      0.60         5
          6       0.59      0.91      0.71        11
          7       1.00      1.00      1.00         1
          8       0.97      0.97      0.97        70
          9       0.80      1.00      0.89         8
         10       1.00      0.50      0.67         2
         11       0.00      0.00      0.00         1
         12       0.85      0.93      0.89        30
         13       0.00      0.00      0.00         1
         14       0.43      1.00      0.60         3
         15       0.67      0.67      0.67        18
         16       0.00      0.00      0.00         1
         17       1.00      0.25      0.40   

/home/enxie/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [1]:
'''
随机从X_test中选择第i个样本做预测，直观的看结果
'''

i=15
ebay_real = X_test[i:i+1]
print("ebay_category_      :",ebay_real[0])
real = df_finn.query('leaf_categ_name=="{}"'.format(ebay_real[0])).values[0][1]
print("gg_category_real_   :",real)
index = pipeline.predict(ebay_real)[0]
print("gg_category_predict_:",d[index])


NameError: name 'X_test' is not defined